# Notebook 02: Patrones de Produccion para AI/ML

**Notebook 7 de la serie Python Extra Class** | Tiempo estimado: 90-120 min

---

## Objetivos de Aprendizaje

Al completar este notebook seras capaz de:

1. **Implementar context managers** para gestion segura de recursos (timers, archivos temporales, estado)
2. **Usar generadores** para procesamiento eficiente de datos grandes sin agotar memoria
3. **Configurar logging estructurado** para observabilidad en produccion
4. **Combinar patrones** en pipelines robustos de AI/ML

---

### Contexto

Este notebook **combina los mejores patrones de los ejemplos ejecutables del curso**:

| Ejemplo | Patron | Lo que aprendimos |
|---------|--------|--------------------|
| `01_excepciones.py` | Excepciones personalizadas | Errores con contexto y metadata |
| `02_context_managers.py` | Context managers | Gestion segura de recursos |
| `03_generadores.py` | Generadores | Procesamiento lazy y eficiente |
| `04_logging_config.py` | Logging estructurado | Observabilidad en produccion |
| `05_comprehensions.py` | Comprehensions | Rendimiento y legibilidad |
| `06_combinados.py` | Pipelines completos | Integracion de patrones |

Aqui los integramos en un flujo realista de produccion para AI/ML.

> **Requisitos:** Solo biblioteca estandar de Python. Sin dependencias externas.

In [ ]:
# =============================================================
# SETUP - Importaciones y verificacion del entorno
# =============================================================

import sys
import time
import logging
import tempfile
import os
import io
import json
from contextlib import contextmanager
from typing import Iterator, Generator, Any, Optional
from collections import defaultdict
from datetime import datetime

print("=" * 60)
print("SETUP COMPLETO - Patrones de Produccion")
print(f"Python: {sys.version}")
print(f"Fecha:  {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("=" * 60)

---

## Seccion 1: Context Managers - Gestion de Recursos

Un **context manager** garantiza que los recursos se inicialicen y se liberen
correctamente, incluso si ocurre una excepcion.

### Flujo de ejecucion

```
with recurso as r:         # __enter__() se ejecuta
    |                      #
    |  codigo del bloque   # Tu codigo aqui
    |                      #
    v                      # __exit__() se ejecuta SIEMPRE
                           #   - sin error: exc_type = None
                           #   - con error: exc_type = tipo de excepcion
```

### Diagrama: Ciclo de vida

```
  __enter__()          Bloque with          __exit__()
 +-----------+      +---------------+     +-----------+
 | Adquirir  | ---> | Usar recurso  | --> | Liberar   |
 | recurso   |      | (tu codigo)   |     | recurso   |
 +-----------+      +---------------+     +-----------+
       |                   |                    |
       |              Si hay error:             |
       |              exc_type != None          |
       |                   |                    |
       +------- GARANTIZADO: siempre pasa ------+
```

### Por que importa en produccion

| Recurso | Si no se limpia... | Solucion con context manager |
|---------|-------------------|------------------------------|
| Archivo | Datos corruptos, file locks | `with open(...)` cierra siempre |
| Conexion DB | Connection pool agotado | `with db.connect()` libera conexion |
| Timer | No sabes que es lento | `with Timer()` mide automaticamente |
| Estado temporal | Config corrupta permanente | `with TemporaryState()` restaura |
| Directorio temp | Disco lleno en servidor | `with tempdir()` elimina archivos |

In [ ]:
# =============================================================
# Context Manager basado en clase: Timer
# =============================================================

class Timer:
    """Mide el tiempo de ejecucion de un bloque de codigo.
    
    Uso:
        with Timer("mi operacion") as t:
            # codigo a medir
        print(t.elapsed)  # tiempo en segundos
    """
    
    def __init__(self, label: str = "Operacion"):
        self.label = label
        self.elapsed: float = 0.0
    
    def __enter__(self):
        self._start = time.perf_counter()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.elapsed = time.perf_counter() - self._start
        status = "ERROR" if exc_type else "OK"
        print(f"[Timer] {self.label}: {self.elapsed:.4f}s ({status})")
        return False  # No suprimimos excepciones


# --- Demo: Comparar list comprehension vs loop ---

N = 500_000

with Timer("List comprehension") as t1:
    resultado_comp = [x ** 2 for x in range(N)]

with Timer("Loop tradicional") as t2:
    resultado_loop = []
    for x in range(N):
        resultado_loop.append(x ** 2)

print(f"\nComparacion:")
print(f"  Comprehension: {t1.elapsed:.4f}s")
print(f"  Loop:          {t2.elapsed:.4f}s")
if t2.elapsed > 0:
    ratio = t2.elapsed / t1.elapsed
    print(f"  Ratio:         {ratio:.2f}x mas lento el loop")
print(f"  Resultados iguales: {resultado_comp == resultado_loop}")

In [ ]:
# =============================================================
# Context Manager: TemporaryState
# Guarda y restaura atributos de un objeto
# =============================================================

class TemporaryState:
    """Temporalmente modifica atributos de un objeto y los restaura al salir.
    
    Ideal para cambiar configuracion de modelos durante inferencia/evaluacion.
    """
    
    def __init__(self, obj, **overrides):
        self.obj = obj
        self.overrides = overrides
        self._saved: dict = {}
    
    def __enter__(self):
        for key, value in self.overrides.items():
            self._saved[key] = getattr(self.obj, key)
            setattr(self.obj, key, value)
        return self.obj
    
    def __exit__(self, *args):
        for key, value in self._saved.items():
            setattr(self.obj, key, value)
        return False


# --- Demo: Configuracion temporal de modelo ---

class ModelConfig:
    """Simulacion de configuracion de un modelo LLM."""
    def __init__(self):
        self.temperature = 0.7
        self.max_tokens = 1024
        self.top_p = 0.9
        self.model_name = "gpt-4"
    
    def __repr__(self):
        return (f"ModelConfig(temperature={self.temperature}, "
                f"max_tokens={self.max_tokens}, top_p={self.top_p})")


config = ModelConfig()
print(f"ANTES:   {config}")

# Temporalmente cambiar para evaluacion (temperatura 0 = deterministico)
with TemporaryState(config, temperature=0.0, max_tokens=256) as cfg:
    print(f"DURANTE: {cfg}")
    # Aqui harias la evaluacion del modelo...
    assert cfg.temperature == 0.0
    assert cfg.max_tokens == 256

print(f"DESPUES: {config}")

# Verificar que se restauro correctamente
assert config.temperature == 0.7
assert config.max_tokens == 1024
print("\nEstado restaurado correctamente!")

In [ ]:
# =============================================================
# Context Manager con @contextmanager (basado en generador)
# =============================================================
# Mas conciso que la version con clase. Ideal para casos simples.

@contextmanager
def timer(label: str = "Operacion"):
    """Version con decorador del Timer. Mas conciso."""
    start = time.perf_counter()
    elapsed = {"value": 0.0}  # dict mutable para poder modificar desde fuera
    try:
        yield elapsed
    finally:
        elapsed["value"] = time.perf_counter() - start
        print(f"[timer] {label}: {elapsed['value']:.4f}s")


@contextmanager
def temporary_directory(prefix: str = "ml_pipeline_"):
    """Crea un directorio temporal y lo elimina al salir.
    
    Util para: checkpoints temporales, datos intermedios, cache de inferencia.
    """
    tmpdir = tempfile.mkdtemp(prefix=prefix)
    print(f"[tmpdir] Creado: {tmpdir}")
    try:
        yield tmpdir
    finally:
        # Limpiar todos los archivos y el directorio
        for f in os.listdir(tmpdir):
            os.remove(os.path.join(tmpdir, f))
        os.rmdir(tmpdir)
        print(f"[tmpdir] Eliminado: {tmpdir}")


# --- Demo: Ambos context managers en accion ---

print("=== Comparacion: clase vs decorador ===")
print()

# Version clase
with Timer("clase") as t:
    sum(range(100_000))

# Version decorador
with timer("decorador") as t:
    sum(range(100_000))

print()
print("=== Directorio temporal ===")
print()

with temporary_directory("checkpoint_") as tmpdir:
    # Simular guardar un checkpoint
    checkpoint_file = os.path.join(tmpdir, "model_epoch_5.json")
    with open(checkpoint_file, "w") as f:
        json.dump({"epoch": 5, "loss": 0.023, "accuracy": 0.97}, f)
    
    print(f"  Archivo creado: {os.path.basename(checkpoint_file)}")
    print(f"  Contenido dir: {os.listdir(tmpdir)}")
    print(f"  Existe: {os.path.exists(checkpoint_file)}")

# Despues del with, el directorio ya no existe
print(f"  Existe despues: {os.path.exists(tmpdir)}")

---

## Seccion 2: Generadores para Procesamiento Eficiente

Los **generadores** producen valores uno a la vez (lazy evaluation), en lugar
de cargar todo en memoria. Esto es critico cuando trabajas con datasets grandes
en ML.

### Lista vs Generador: Uso de memoria

```
  LISTA (carga todo en memoria):
  +-----+-----+-----+-----+-----+-----+-----+-----+
  | r0  | r1  | r2  | r3  | r4  | ... | rN-1| rN  |  <-- TODO en RAM
  +-----+-----+-----+-----+-----+-----+-----+-----+
  Memoria: O(N)

  GENERADOR (produce uno a la vez):
  +-----+
  | r_i | --> procesar --> siguiente
  +-----+
  Memoria: O(1)
```

### Por que importa en AI/ML

```
  Dataset de 10GB de textos para entrenar un LLM:

  MAL (lista):     cargar_todo() --> [10GB en RAM] --> OOM Error!
  BIEN (generador): stream()     --> [1 batch]     --> procesar --> siguiente
```

| Caso de uso | Sin generador | Con generador |
|-------------|---------------|---------------|
| Leer 1M de registros | `list(read_all())` = 8GB RAM | `yield` = ~KB RAM |
| Batches de training | `all_batches = [...]` | `yield batch` |
| Paginacion de API | Cargar todas las paginas | `yield page` |
| Preprocesar texto | Lista de documentos | Stream de documentos |

In [ ]:
# =============================================================
# Generadores: batch_iterator
# =============================================================

def batch_iterator(data, batch_size: int = 32) -> Generator:
    """Divide datos en batches del tamano especificado.
    
    Patron fundamental en ML: DataLoader, training loops, inferencia batch.
    """
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]


# --- Demo ---

# Simular 1000 muestras de entrenamiento
dataset = [f"muestra_{i}" for i in range(1000)]

print(f"Dataset: {len(dataset)} muestras")
print(f"Batch size: 100")
print()

# Iterar por batches
batches = list(batch_iterator(dataset, batch_size=100))
print(f"Total batches: {len(batches)}")
print(f"Tamano primer batch: {len(batches[0])}")
print(f"Primeros 3 del batch 0: {batches[0][:3]}")
print(f"Primeros 3 del batch 5: {batches[5][:3]}")
print()

# Uso tipico en un training loop
print("Simulando training loop:")
for epoch in range(2):
    total = 0
    for batch_idx, batch in enumerate(batch_iterator(dataset, 100)):
        total += len(batch)
    print(f"  Epoch {epoch + 1}: {batch_idx + 1} batches, {total} muestras procesadas")

In [ ]:
# =============================================================
# Comparacion de memoria: Lista vs Generador
# =============================================================

def generate_records(n: int) -> Generator:
    """Genera n registros simulados de ML (lazy)."""
    for i in range(n):
        yield {"id": i, "score": i * 0.001, "label": i % 10}


def sliding_window(data, window_size: int = 3) -> Generator:
    """Genera ventanas deslizantes sobre una secuencia.
    
    Util para: series temporales, n-gramas en NLP, features secuenciales.
    """
    for i in range(len(data) - window_size + 1):
        yield data[i:i + window_size]


# --- Comparacion de memoria ---

N = 100_000

# Enfoque 1: Lista completa
with Timer("Lista completa"):
    lista = [{'id': i, 'score': i * 0.001, 'label': i % 10} for i in range(N)]
    size_lista = sys.getsizeof(lista)

# Enfoque 2: Generador
with Timer("Generador (creacion)"):
    gen = generate_records(N)
    size_gen = sys.getsizeof(gen)

print()
print("+" + "-" * 42 + "+")
print(f"| {'Metodo':<20} | {'Memoria':>18} |")
print("+" + "-" * 42 + "+")
print(f"| {'Lista (100K recs)':<20} | {size_lista:>14,} bytes |")
print(f"| {'Generador':<20} | {size_gen:>14,} bytes |")
print("+" + "-" * 42 + "+")
if size_gen > 0:
    print(f"| {'Ratio':<20} | {size_lista / size_gen:>15.0f}x mas |")
    print("+" + "-" * 42 + "+")

# Limpiar
del lista

# --- Demo: Sliding window ---
print()
print("=== Sliding Window (ventana deslizante) ===")
serie_temporal = [10, 20, 30, 40, 50, 60, 70, 80]
print(f"Datos: {serie_temporal}")
print(f"Ventanas (tamano=3):")
for i, ventana in enumerate(sliding_window(serie_temporal, 3)):
    promedio = sum(ventana) / len(ventana)
    print(f"  Ventana {i}: {ventana} -> promedio movil: {promedio:.1f}")

In [ ]:
# =============================================================
# Pipeline de Generadores: ETL encadenado
# =============================================================
# Cada funcion es un generador que consume del anterior.
# Los datos fluyen registro por registro, sin cargar todo en memoria.

def read_records(raw_data: list) -> Generator:
    """Etapa 1: Leer y limpiar registros crudos."""
    for line in raw_data:
        yield line.strip()


def filter_valid(records: Generator) -> Generator:
    """Etapa 2: Filtrar registros vacios y comentarios."""
    for r in records:
        if r and not r.startswith("#"):
            yield r


def parse_json_records(records: Generator) -> Generator:
    """Etapa 3: Parsear JSON. Omitir registros malformados."""
    for r in records:
        try:
            yield json.loads(r)
        except json.JSONDecodeError:
            continue  # Saltar registros invalidos


def enrich(records: Generator) -> Generator:
    """Etapa 4: Enriquecer con metadata de procesamiento."""
    for r in records:
        r["processed_at"] = datetime.now().isoformat()
        r["pipeline_version"] = "1.0"
        yield r


# --- Datos de ejemplo (simulan lecturas de un archivo/API) ---

raw_data = [
    '  {"user": "alice", "score": 0.95, "model": "bert"}  ',
    '# Este es un comentario',
    '',
    '{"user": "bob", "score": 0.82, "model": "gpt-4"}',
    'esto no es JSON valido!!!',
    '{"user": "carol", "score": 0.91, "model": "bert"}',
    '  ',
    '{"user": "dave", "score": 0.78, "model": "gpt-4"}',
    '# Otro comentario',
    '{"user": "eve", "score": 0.99, "model": "claude"}',
]

# --- Encadenar el pipeline ---

print("=== Pipeline de Generadores ===")
print(f"Registros crudos: {len(raw_data)}")
print()

# Crear la cadena (lazy - nada se ejecuta aun!)
pipeline = enrich(
    parse_json_records(
        filter_valid(
            read_records(raw_data)
        )
    )
)

# Ahora si consumimos el pipeline
resultados = []
for i, record in enumerate(pipeline):
    resultados.append(record)
    print(f"  Registro {i + 1}: {record['user']:>6} | "
          f"score={record['score']:.2f} | "
          f"modelo={record['model']}")

print(f"\nRegistros procesados exitosamente: {len(resultados)} de {len(raw_data)}")
print(f"Filtrados/invalidos: {len(raw_data) - len(resultados)}")

---

## Seccion 3: Logging Estructurado

El modulo `logging` de Python es la forma estandar de registrar eventos en produccion.
**Nunca uses `print()` en codigo de produccion.** Logging te da niveles, formato,
destinos multiples y filtrado.

### Los 5 niveles de logging

```
 Nivel      | Valor | Cuando usarlo
 -----------|-------|------------------------------------------
 DEBUG      |  10   | Detalle tecnico para diagnostico
 INFO       |  20   | Confirmacion de que todo funciona bien
 WARNING    |  30   | Algo inesperado, pero el programa continua
 ERROR      |  40   | Error serio, una funcion fallo
 CRITICAL   |  50   | Error fatal, el programa no puede continuar
```

### Guia de decision

```
 Quieres registrar...              --> Usa
 -------------------------------------------------------
 Valor de variables para debug     --> logger.debug()
 "Pipeline iniciado", "Batch OK"   --> logger.info()
 Dato faltante pero hay default    --> logger.warning()
 Fallo al procesar un registro     --> logger.error()
 DB caida, sin disco, OOM          --> logger.critical()
 Excepcion con traceback           --> logger.exception()
```

### Jerarquia de loggers

```
  root logger
  +-- ml_pipeline                 (logger padre)
  |   +-- ml_pipeline.loader      (logger hijo - hereda config)
  |   +-- ml_pipeline.model       (logger hijo)
  |   +-- ml_pipeline.evaluator   (logger hijo)
```

In [ ]:
# =============================================================
# Logging profesional para notebooks
# =============================================================
# Usamos un StringIO handler para capturar la salida en el notebook.

def setup_logger(name: str, level=logging.DEBUG) -> tuple:
    """Configura un logger que escribe a un StringIO (visible en notebook).
    
    Returns:
        (logger, stream) - el logger y el stream para leer la salida
    """
    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.handlers.clear()  # Evitar duplicados en re-ejecuciones
    
    stream = io.StringIO()
    handler = logging.StreamHandler(stream)
    handler.setLevel(level)
    
    formatter = logging.Formatter(
        "%(asctime)s | %(levelname)-8s | %(name)s | %(message)s",
        datefmt="%H:%M:%S"
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    return logger, stream


# --- Demo: Los 5 niveles ---

logger, stream = setup_logger("ml_pipeline.demo")

logger.debug("Cargando dataset desde /data/train.jsonl")
logger.info("Pipeline iniciado con %d registros", 10000)  # Lazy formatting (CORRECTO)
logger.warning("Campo 'label' faltante en 23 registros, usando default=0")
logger.error("Fallo al conectar con API de embeddings: timeout 30s")
logger.critical("Sin espacio en disco: /models/ esta lleno")

# Logging de excepcion (captura traceback automaticamente)
try:
    result = 1 / 0
except ZeroDivisionError:
    logger.exception("Error en calculo de metricas")

# Mostrar toda la salida
print("=== Salida del Logger ===")
print(stream.getvalue())

# --- Anti-patron vs patron correcto ---
print("=" * 50)
print("IMPORTANTE: Lazy formatting")
print()
print("CORRECTO (lazy - solo formatea si el nivel esta activo):")
print('  logger.info("Procesados %d de %d", count, total)')
print()
print("INCORRECTO (f-string - siempre formatea, desperdicia CPU):")
print('  logger.info(f"Procesados {count} de {total}")  # NO!')

In [ ]:
# =============================================================
# Logging Estructurado: Formato JSON
# =============================================================
# En produccion, los logs en JSON son parseables por herramientas
# como ELK, Datadog, CloudWatch, etc.

class JSONFormatter(logging.Formatter):
    """Formatea logs como JSON estructurado.
    
    Ventajas:
    - Parseable automaticamente por herramientas de monitoreo
    - Busquedas por campo (level=ERROR, module=loader)
    - Alertas automaticas basadas en patrones
    - Dashboards en tiempo real
    """
    
    def format(self, record: logging.LogRecord) -> str:
        log_data = {
            "timestamp": self.formatTime(record),
            "level": record.levelname,
            "message": record.getMessage(),
            "module": record.module,
            "function": record.funcName,
            "line": record.lineno,
        }
        if record.exc_info and record.exc_info[0]:
            log_data["exception"] = self.formatException(record.exc_info)
        # Campos extra personalizados
        if hasattr(record, "extra_data"):
            log_data["extra"] = record.extra_data
        return json.dumps(log_data, ensure_ascii=False)


# --- Setup con JSON formatter ---

json_logger = logging.getLogger("ml_pipeline.json")
json_logger.setLevel(logging.DEBUG)
json_logger.handlers.clear()

json_stream = io.StringIO()
json_handler = logging.StreamHandler(json_stream)
json_handler.setFormatter(JSONFormatter())
json_logger.addHandler(json_handler)

# --- Simular eventos de un pipeline ---

json_logger.info("Pipeline iniciado")
json_logger.info("Cargados %d registros en %.2fs", 50000, 1.23)
json_logger.warning("Batch 42 tiene %d registros vacios", 15)

try:
    raise ValueError("Score fuera de rango: -0.5")
except ValueError:
    json_logger.exception("Error en validacion")

json_logger.info("Pipeline completado: %d registros procesados", 49985)

# --- Mostrar salida ---

print("=== Logs en formato JSON ===")
print()
for line in json_stream.getvalue().strip().split("\n"):
    # Pretty print cada linea JSON
    parsed = json.loads(line)
    print(json.dumps(parsed, indent=2, ensure_ascii=False))
    print()

---

## Seccion 4: Combinando Patrones - Mini Pipeline de Produccion

Ahora integramos **todos los patrones** en un pipeline realista:

```
 +------------------+    +------------------+    +------------------+
 | 1. INGESTION     |    | 2. VALIDACION    |    | 3. TRANSFORMACION|
 |                  |    |                  |    |                  |
 | - Context mgr    |--->| - Excepciones    |--->| - Generadores   |
 |   (timer)        |    |   personalizadas |    |   (lazy ETL)    |
 | - Logging INFO   |    | - Logging WARN   |    | - Logging DEBUG |
 +------------------+    +------------------+    +------------------+
          |                       |                       |
          v                       v                       v
 +------------------+    +------------------+    +------------------+
 | 4. AGREGACION    |    | 5. REPORTE       |    | 6. CLEANUP       |
 |                  |    |                  |    |                  |
 | - Comprehensions |--->| - Estadisticas   |--->| - Context mgr   |
 | - defaultdict    |    | - Logging INFO   |    |   (temp files)  |
 +------------------+    +------------------+    +------------------+
```

### Patrones integrados

- **Excepciones personalizadas**: errores con contexto y metadata
- **Context managers**: Timer para medir cada fase, temp dirs para cleanup
- **Generadores**: procesamiento lazy registro por registro
- **Logging**: observabilidad en cada etapa
- **Comprehensions**: agregacion eficiente de resultados

In [ ]:
# =============================================================
# Mini Pipeline de Produccion: Todos los patrones combinados
# =============================================================

# --- Excepciones personalizadas ---

class PipelineError(Exception):
    """Error base del pipeline."""
    def __init__(self, message: str, stage: str, details: Optional[dict] = None):
        super().__init__(message)
        self.stage = stage
        self.details = details or {}


class ValidationError(PipelineError):
    """Error de validacion de datos."""
    pass


class TransformError(PipelineError):
    """Error en la transformacion de datos."""
    pass


# --- Pipeline completo ---

class DataPipeline:
    """Pipeline de procesamiento de datos con todos los patrones de produccion."""
    
    def __init__(self, name: str = "default"):
        self.name = name
        self.stats = defaultdict(int)
        self.timings = {}
        
        # Setup logger
        self.logger = logging.getLogger(f"pipeline.{name}")
        self.logger.setLevel(logging.DEBUG)
        self.logger.handlers.clear()
        
        self._log_stream = io.StringIO()
        handler = logging.StreamHandler(self._log_stream)
        handler.setFormatter(logging.Formatter(
            "%(asctime)s | %(levelname)-8s | %(message)s",
            datefmt="%H:%M:%S"
        ))
        self.logger.addHandler(handler)
    
    @contextmanager
    def _timed_stage(self, stage_name: str):
        """Context manager que mide y registra el tiempo de cada etapa."""
        self.logger.info("Etapa '%s' iniciada", stage_name)
        start = time.perf_counter()
        try:
            yield
        except PipelineError:
            raise  # Re-lanzar errores del pipeline
        except Exception as e:
            self.logger.exception("Error inesperado en etapa '%s'", stage_name)
            raise PipelineError(str(e), stage=stage_name) from e
        finally:
            elapsed = time.perf_counter() - start
            self.timings[stage_name] = elapsed
            self.logger.info("Etapa '%s' completada en %.4fs", stage_name, elapsed)
    
    def _validate(self, records: Generator) -> Generator:
        """Generador: valida cada registro."""
        for record in records:
            self.stats["total"] += 1
            
            # Validar campos requeridos
            required = ["user", "score"]
            missing = [f for f in required if f not in record]
            if missing:
                self.stats["invalid"] += 1
                self.logger.warning(
                    "Registro %d: campos faltantes %s - omitido",
                    self.stats["total"], missing
                )
                continue
            
            # Validar rango de score
            if not (0.0 <= record["score"] <= 1.0):
                self.stats["invalid"] += 1
                self.logger.warning(
                    "Registro %d: score=%.2f fuera de [0,1] - omitido",
                    self.stats["total"], record["score"]
                )
                continue
            
            self.stats["valid"] += 1
            yield record
    
    def _transform(self, records: Generator) -> Generator:
        """Generador: transforma cada registro."""
        for record in records:
            record["score_pct"] = round(record["score"] * 100, 1)
            record["category"] = (
                "excelente" if record["score"] >= 0.9
                else "bueno" if record["score"] >= 0.7
                else "regular"
            )
            record["processed_at"] = datetime.now().isoformat()
            self.stats["transformed"] += 1
            self.logger.debug("Transformado: %s -> %s", record["user"], record["category"])
            yield record
    
    def run(self, raw_data: list) -> list:
        """Ejecuta el pipeline completo."""
        self.logger.info("=" * 50)
        self.logger.info("Pipeline '%s' iniciado con %d registros crudos", self.name, len(raw_data))
        
        results = []
        
        # Etapa 1: Ingestion
        with self._timed_stage("ingestion"):
            parsed = []
            for line in raw_data:
                line = line.strip()
                if not line or line.startswith("#"):
                    self.stats["skipped"] += 1
                    continue
                try:
                    parsed.append(json.loads(line))
                except json.JSONDecodeError:
                    self.stats["parse_errors"] += 1
                    self.logger.warning("JSON invalido: %s", line[:50])
        
        # Etapa 2: Validacion + Transformacion (generadores encadenados)
        with self._timed_stage("validacion_y_transformacion"):
            pipeline = self._transform(self._validate(iter(parsed)))
            results = list(pipeline)
        
        # Etapa 3: Agregacion
        with self._timed_stage("agregacion"):
            por_categoria = defaultdict(list)
            for r in results:
                por_categoria[r["category"]].append(r["user"])
            
            self.stats["output"] = len(results)
            for cat, users in por_categoria.items():
                self.logger.info("Categoria '%s': %d registros", cat, len(users))
        
        # Reporte final
        self.logger.info("=" * 50)
        self.logger.info("PIPELINE COMPLETADO")
        for key, val in sorted(self.stats.items()):
            self.logger.info("  %-20s: %d", key, val)
        
        total_time = sum(self.timings.values())
        self.logger.info("  %-20s: %.4fs", "TIEMPO_TOTAL", total_time)
        
        return results
    
    def get_logs(self) -> str:
        """Retorna todos los logs generados."""
        return self._log_stream.getvalue()


# --- Ejecutar con datos de ejemplo ---

sample_data = [
    '{"user": "alice", "score": 0.95, "model": "bert"}',
    '{"user": "bob", "score": 0.82, "model": "gpt-4"}',
    '# Comentario ignorado',
    '',
    '{"user": "carol", "score": 0.91, "model": "bert"}',
    'esto no es JSON',
    '{"user": "dave", "score": 1.5, "model": "gpt-4"}',
    '{"user": "eve", "score": 0.99, "model": "claude"}',
    '{"score": 0.50}',
    '{"user": "frank", "score": 0.65, "model": "bert"}',
    '{"user": "grace", "score": 0.88, "model": "claude"}',
    '{"user": "hank", "score": -0.1, "model": "gpt-4"}',
]

pipeline = DataPipeline(name="evaluacion_modelos")
results = pipeline.run(sample_data)

print("=== LOG COMPLETO DEL PIPELINE ===")
print(pipeline.get_logs())

print("=== RESULTADOS ===")
for r in results:
    print(f"  {r['user']:>8} | score={r['score_pct']:>5}% | {r['category']}")

In [ ]:
# =============================================================
# Pipeline manejando errores criticos
# =============================================================
# Demostrar como el pipeline maneja datos problematicos de forma
# elegante: warnings para errores recuperables, excepciones para criticos.

print("=== Caso 1: Errores recuperables (el pipeline continua) ===")
print()

datos_mixtos = [
    '{"user": "ok_user", "score": 0.85}',       # OK
    '{malformado!!!}',                              # Parse error (warning)
    '{"user": "sin_score"}',                       # Validacion falla (warning)
    '{"user": "score_malo", "score": 999}',       # Score fuera de rango (warning)
    '{"user": "otro_ok", "score": 0.72}',         # OK
]

p1 = DataPipeline(name="test_recuperable")
resultados = p1.run(datos_mixtos)
print(p1.get_logs())
print(f"Resultado: {len(resultados)} registros procesados de {len(datos_mixtos)} crudos")

print()
print("=" * 60)
print()

print("=== Caso 2: Error critico en una etapa (pipeline se detiene limpiamente) ===")
print()

class StrictPipeline(DataPipeline):
    """Pipeline que falla si hay demasiados errores."""
    
    def __init__(self, name: str, max_error_rate: float = 0.5):
        super().__init__(name)
        self.max_error_rate = max_error_rate
    
    def run(self, raw_data: list) -> list:
        """Ejecuta y verifica tasa de error."""
        results = super().run(raw_data)
        
        total = self.stats.get("total", 0)
        invalid = self.stats.get("invalid", 0)
        
        if total > 0:
            error_rate = invalid / total
            if error_rate > self.max_error_rate:
                raise ValidationError(
                    f"Tasa de error {error_rate:.0%} excede limite {self.max_error_rate:.0%}",
                    stage="post_validacion",
                    details={"error_rate": error_rate, "total": total, "invalid": invalid}
                )
        
        return results


# Datos donde la mayoria son invalidos
datos_malos = [
    '{"user": "unico_bueno", "score": 0.9}',
    '{"sin_campos": true}',
    '{"user": "x", "score": -5}',
    '{"user": "y", "score": 100}',
    '{"otra_cosa": 123}',
]

p2 = StrictPipeline(name="test_critico", max_error_rate=0.5)

try:
    resultados = p2.run(datos_malos)
except ValidationError as e:
    print(p2.get_logs())
    print(f"EXCEPCION CAPTURADA:")
    print(f"  Tipo:     {type(e).__name__}")
    print(f"  Mensaje:  {e}")
    print(f"  Etapa:    {e.stage}")
    print(f"  Detalles: {e.details}")
    print()
    print("El pipeline se detuvo limpiamente con toda la informacion necesaria.")
    print("Los timings y logs ya estan registrados para diagnostico.")

---

## Cuando usar cada patron

| Situacion | Patron | Ejemplo concreto |
|-----------|--------|------------------|
| Recurso que necesita cleanup | **Context Manager** | Conexiones DB, archivos temporales, timers, locks |
| Datos grandes o streaming | **Generador** | Archivos de log, paginacion de API, batches de training |
| Observabilidad en produccion | **Logging** | Cada modulo, cada error, eventos clave del pipeline |
| Errores con contexto | **Excepciones personalizadas** | Errores de dominio con metadata (stage, details) |
| Transformacion de colecciones | **Comprehensions** | Filtrado, mapeo, agregacion de datos |

### Reglas de oro

1. **Si adquieres un recurso, usa `with`** - nunca confies en que tu codigo llegara al `close()`
2. **Si procesas muchos datos, usa `yield`** - la memoria es finita, los generadores no
3. **Si esta en produccion, usa `logging`** - `print()` es para notebooks, no para servidores
4. **Si el error tiene contexto, crea una excepcion** - `ValueError` no dice que paso ni donde
5. **Si combinas patrones, hazlo en capas** - cada capa tiene una responsabilidad clara

---

## Ejercicios

Completa los siguientes ejercicios para practicar los patrones de produccion.
Cada ejercicio indica que patrones debes usar.

In [ ]:
# =============================================================
# EJERCICIO 1: Context Manager - DatabaseConnection
# =============================================================
#
# Crea un context manager (basado en clase) llamado DatabaseConnection que:
#
# 1. __init__: recibe host (str) y database (str)
# 2. __enter__: 
#    - Simula conectar (print "Conectando a {host}/{database}")
#    - Inicializa una lista self.operations = [] para rastrear operaciones
#    - Retorna self
# 3. Metodo execute(query): agrega el query a self.operations
# 4. __exit__:
#    - Si NO hubo excepcion: print "COMMIT: {n} operaciones"
#    - Si SI hubo excepcion: print "ROLLBACK por: {tipo_error}"
#    - Siempre: print "Desconectando de {host}/{database}"
#    - Retorna False (no suprimir excepciones)
#
# Prueba con:
#   with DatabaseConnection("localhost", "ml_models") as db:
#       db.execute("INSERT INTO results VALUES (0.95, 'bert')")
#       db.execute("UPDATE models SET status='active'")
#   # Deberia mostrar COMMIT
#
#   with DatabaseConnection("localhost", "ml_models") as db:
#       db.execute("INSERT INTO results VALUES (0.8, 'gpt')")
#       raise ValueError("Score invalido")
#   # Deberia mostrar ROLLBACK

class DatabaseConnection:
    pass


# Descomenta para probar:
# with DatabaseConnection("localhost", "ml_models") as db:
#     db.execute("INSERT INTO results VALUES (0.95, 'bert')")
#     db.execute("UPDATE models SET status='active'")

In [ ]:
# =============================================================
# EJERCICIO 2: Pipeline de Generadores con Logging
# =============================================================
#
# Crea un pipeline de generadores para procesar predicciones de un modelo.
# Cada funcion debe ser un generador (usar yield).
#
# Funciones a implementar:
#
# 1. leer_predicciones(datos: list) -> Generator:
#    - Recibe una lista de strings JSON
#    - Parsea cada uno con json.loads
#    - Si falla el parseo, loguea WARNING y continua
#    - Yield de cada dict parseado
#
# 2. filtrar_confianza(preds: Generator, umbral: float = 0.8) -> Generator:
#    - Solo yield de predicciones donde pred["confidence"] >= umbral
#    - Loguea DEBUG para cada prediccion filtrada
#
# 3. agrupar_por_batch(preds: Generator, batch_size: int = 3) -> Generator:
#    - Acumula predicciones en batches de tamano batch_size
#    - Yield de cada batch (lista)
#    - No olvidar el ultimo batch si es mas pequeno
#
# Datos de prueba:
datos_predicciones = [
    '{"id": 1, "label": "gato", "confidence": 0.95}',
    '{"id": 2, "label": "perro", "confidence": 0.60}',
    '{"id": 3, "label": "gato", "confidence": 0.88}',
    '{INVALIDO}',
    '{"id": 5, "label": "pajaro", "confidence": 0.92}',
    '{"id": 6, "label": "perro", "confidence": 0.45}',
    '{"id": 7, "label": "gato", "confidence": 0.99}',
    '{"id": 8, "label": "pajaro", "confidence": 0.81}',
]
#
# Encadenar: leer -> filtrar -> agrupar
# Imprimir cada batch resultante.

def leer_predicciones(datos: list) -> Generator:
    pass


def filtrar_confianza(preds, umbral: float = 0.8) -> Generator:
    pass


def agrupar_por_batch(preds, batch_size: int = 3) -> Generator:
    pass


# Descomenta para probar:
# pipeline = agrupar_por_batch(filtrar_confianza(leer_predicciones(datos_predicciones)))
# for batch in pipeline:
#     print(f"Batch: {batch}")

In [ ]:
# =============================================================
# EJERCICIO 3: Mini-ETL Completo (todos los patrones)
# =============================================================
#
# Construye un mini pipeline ETL que combine TODOS los patrones:
#
# Clase: MLExperimentPipeline
#
# __init__(self, experiment_name: str):
#   - Configurar logger con nombre "experiment.{experiment_name}"
#   - Inicializar self.timings = {} y self.stats = defaultdict(int)
#
# Metodo context manager (con @contextmanager):
#   timed_phase(self, phase_name: str):
#   - Mide tiempo de cada fase
#   - Loguea inicio y fin con tiempo
#   - Guarda timing en self.timings
#
# Generador:
#   process_results(self, raw_results: list) -> Generator:
#   - Parsea JSON, valida campos ("model", "accuracy", "dataset")
#   - Filtra accuracy < 0 o > 1
#   - Enriquece con timestamp y experiment_name
#   - Loguea cada paso, cuenta stats
#
# Metodo principal:
#   run(self, raw_data: list) -> dict:
#   - Fase "ingestion": parsear datos con timed_phase
#   - Fase "processing": usar generador con timed_phase
#   - Fase "analysis": calcular mejor modelo, promedio por modelo
#   - Retorna dict con "results", "stats", "timings", "best_model"
#
# Datos de prueba:
datos_experimento = [
    '{"model": "bert", "accuracy": 0.92, "dataset": "squad"}',
    '{"model": "gpt-4", "accuracy": 0.95, "dataset": "squad"}',
    '{"model": "bert", "accuracy": 0.88, "dataset": "mnli"}',
    '{INVALIDO}',
    '{"model": "claude", "accuracy": 0.97, "dataset": "squad"}',
    '{"model": "gpt-4", "accuracy": 1.5, "dataset": "error"}',
    '{"model": "claude", "accuracy": 0.93, "dataset": "mnli"}',
]

class MLExperimentPipeline:
    pass


# Descomenta para probar:
# exp = MLExperimentPipeline("benchmark_q1_2026")
# report = exp.run(datos_experimento)
# print(f"Mejor modelo: {report['best_model']}")
# print(f"Timings: {report['timings']}")

---

## Checklist de Consolidacion

Marca cada item cuando lo hayas comprendido y puedas implementarlo sin mirar ejemplos:

- [ ] **Context managers con clase**: implementar `__enter__` y `__exit__`, saber que `__exit__` recibe info de excepcion
- [ ] **Context managers con `@contextmanager`**: usar `yield` en un bloque `try/finally`, saber cuando preferir este enfoque
- [ ] **Generadores con `yield`**: crear funciones generadoras, entender lazy evaluation y ahorro de memoria
- [ ] **Encadenar generadores**: construir pipelines donde cada generador consume del anterior
- [ ] **`batch_iterator`**: dividir datos en batches para procesamiento por lotes
- [ ] **Logging con niveles**: usar DEBUG/INFO/WARNING/ERROR/CRITICAL apropiadamente
- [ ] **Lazy formatting en logging**: usar `%s` y `%d` en lugar de f-strings
- [ ] **JSON formatter**: crear logs estructurados parseables por herramientas de monitoreo
- [ ] **Excepciones personalizadas**: crear jerarquias con metadata (stage, details)
- [ ] **Combinar patrones**: integrar context managers + generadores + logging + excepciones en un pipeline

---

### Resumen de la serie Python Extra Class (7 notebooks)

| Notebook | Tema | Patron clave |
|----------|------|--------------|
| 01 | Fundamentos de Python para AI | Tipos, funciones, estructuras |
| 02 | Estructuras de datos avanzadas | Listas, dicts, sets, comprehensions |
| 03 | Programacion orientada a objetos | Clases, herencia, polimorfismo |
| 04 | Manejo de archivos y datos | I/O, JSON, CSV, paths |
| 05 | Testing y debugging | unittest, assert, pdb |
| 06 | Concurrencia basica | threading, asyncio |
| **07** | **Patrones de produccion** | **Context managers, generadores, logging, pipelines** |

---

**Con estos patrones, estas listo para construir pipelines de AI/ML robustos y mantenibles.**

Los mismos patrones que usamos aqui son los que encontraras en frameworks como:
- **LangChain**: context managers para callbacks, generadores para streaming
- **PyTorch**: DataLoader usa generadores, logging integrado
- **FastAPI**: context managers para lifespan, excepciones HTTP personalizadas
- **MLflow**: context managers para tracking de experimentos

Dominar estos fundamentos te permitira aprender cualquier framework rapidamente
porque ya entiendes los patrones subyacentes.